In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
without_mask = np.load("with_out_mask.npy")
with_mask = np.load("with_mask.npy")

In [ ]:
without_mask.shape

In [ ]:
with_mask.shape

In [ ]:
with_mask = with_mask.reshape(200,50 * 50 * 3)
without_mask = without_mask.reshape(200,50 * 50 * 3)

In [ ]:
with_mask.shape

In [ ]:
without_mask.shape

In [ ]:
x = np.r_[with_mask, without_mask]

In [ ]:
x.shape

In [ ]:
label = np.zeros(x.shape[0])

In [ ]:
label[200:] = 1.0

In [ ]:
names = {0:"Mask", 1:"NoMask"}

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
 x_train,x_test,y_train,y_test = train_test_split(x,label, test_size = 0.20)

In [ ]:
x_train.shape

## Dimensionality reduction

In [ ]:
#PCA
from  sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=4)
x_train = pca.fit_transform(x_train)

In [ ]:
x_train[0]

In [ ]:
x_train.shape

In [ ]:
svm= SVC()
svm.fit(x_train, y_train)

In [ ]:
x_test= pca.transform(x_test)

In [ ]:
y_pred = svm.predict(x_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
haar_data = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
capture = cv2.VideoCapture(0)
data = []
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img =  capture.read()
    if flag:
        faces = haar_data.detectMultiScale(img)
        for x,y,w,h in faces:
            cv2.rectangle(img,(x,y), (x+w, y+h), (0,0,255),3)
            face = img[y:y+h, x:x+w, :]
            face = cv2.resize(face, (50,50))
            face = face.reshape(1,-1)
            face = pca.transform(face)
            pred = svm.predict(face)[0] 
            n = names[int(pred)]
            cv2.putText(img, n, (x,y), font, 1, (255,255,255), 2)
            print(n)
        cv2.imshow('image',img)
        if cv2.waitKey(4) == 27:
            break
capture.release()
cv2.destroyAllWindows()